In [1]:
#loading required libaries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
#Loading the data
data = pd.read_csv("/home/panas/Documents/Programming/DataSets/NewYork.csv")

In [3]:
data.columns

Index(['datetime', 'Humidity', 'Pressure', 'Temperature',
       'Weather Description', 'Wind Dirn', 'Win Speed'],
      dtype='object')

In [3]:
data.head()

,datetime,Humidity,Pressure,Temperature,Weather Description,Wind Dirn,Win Speed
0,2012-10-02 00:00:00,55.0,1012.0,289.040311,few clouds,264,6
1,2012-10-02 01:00:00,54.0,1012.0,289.119575,few clouds,264,6
2,2012-10-02 02:00:00,54.0,1012.0,289.198838,few clouds,265,6
3,2012-10-02 03:00:00,54.0,1012.0,289.278102,few clouds,265,6
4,2012-10-02 04:00:00,53.0,1012.0,289.357365,few clouds,265,6


In [4]:
#Spliting datetime to date and time
new_data = data.copy()
new_data[["Date", "Time"]] = new_data.datetime.str.split(expand=True)

In [5]:
new_data.head()

,datetime,Humidity,Pressure,Temperature,Weather Description,Wind Dirn,Win Speed,Date,Time
0,2012-10-02 00:00:00,55.0,1012.0,289.040311,few clouds,264,6,2012-10-02,00:00:00
1,2012-10-02 01:00:00,54.0,1012.0,289.119575,few clouds,264,6,2012-10-02,01:00:00
2,2012-10-02 02:00:00,54.0,1012.0,289.198838,few clouds,265,6,2012-10-02,02:00:00
3,2012-10-02 03:00:00,54.0,1012.0,289.278102,few clouds,265,6,2012-10-02,03:00:00
4,2012-10-02 04:00:00,53.0,1012.0,289.357365,few clouds,265,6,2012-10-02,04:00:00


In [6]:
#Split date  and drop datetime
new_data[["Year", "Months","Day" ]] = new_data.Date.str.split("-", expand = True)
new_data = new_data.drop(["datetime"], axis = 1)

In [7]:
#Reindex the dataframe in require order

Columns = ["Year", "Months", "Day", 'Time',
        "Temperature", "Humidity", "Pressure", 
        "Wind Dirn", "Win Speed", "Weather Description"]

new_data = new_data.reindex(Columns, axis = 1)

In [8]:
#Convert kelvin  to celcius
new_data["Temperature"]=new_data["Temperature"].apply(lambda x : x-272.15)

In [9]:
#Create int value for obj value weather description
label_encoder = LabelEncoder()
new_data["Weather Description"] = label_encoder.fit_transform(new_data["Weather Description"])

In [10]:
#from sklearn.preprocessing import OneHotEncoder
#hot_encoder = OneHotEncoder()
#weather_encoded = hot_encoder.fit_transform(new_data[["Weather Description"]])

In [11]:
#Filling empty cells with values
for col in Columns[4:]:
    new_data[col].fillna(method= "ffill", inplace = True)

In [12]:
#Grouping the columns value in list

def Group(parameter):  
    total = []
    day = []
    for i in range(0,new_data.shape[0]):
        today = new_data["Day"][i]
        if i == 0:
            prev_day = new_data["Day"][i]
        else:
            prev_day = new_data["Day"][i-1]

        if today == prev_day:
            temp = new_data[parameter][i]
            day.append(round(temp,2))
        else:
            total.append(day)
            day = []
            day.append(round(new_data[parameter][i],2))
    return total

In [13]:
Temp = Group("Temperature")
Humid = Group("Humidity")
Press = Group("Pressure")
WindDirn = Group("Wind Dirn")
WinSpeed = Group("Win Speed")
WeatherDiscpc = Group("Weather Description")
parameter_disc = {
             "Temp": Temp, 
             "Humid": Humid, 
             "Press": Press,
             "WindDirn": WindDirn, 
             "WinSpeed": WinSpeed,
            "WeatherDiscpc": WeatherDiscpc
            }

In [14]:
#Keep only one Row per day

indexes = []
for i in range(1,new_data.shape[0]):
    today = new_data["Day"][i]
    if i == 0:
        prev_day = new_data["Day"][i]
    else:
        prev_day = new_data["Day"][i-1]
    
    if today == prev_day:
        indexes.append(i)
new_data = new_data.drop(indexes)

In [15]:
#Drop the time axis

new_data = new_data.drop(["Time"], axis = 1)
df = new_data.copy()
df = df.drop(44448)

In [16]:
 #Calculate Max, Min, Average value of each day of given parameter
def maxMin(parameter):
    maxm = []
    mini = []
    avg = []
    for col in parameter:
        daymax  = []
        daymin = []
        dayavg = []
        if col != WeatherDiscpc:
            for day in col:
                mx = max(day)
                mn = min(day)
                ag = sum(day)/len(day)
                daymax.append(mx)
                daymin.append(mn)
                dayavg.append(round(ag,2))
            maxm.append(daymax)
            mini.append(daymin)
            avg.append(dayavg)
    return maxm, mini, avg

maxm, minm, avg = maxMin(parameter_disc.values())

In [17]:
#Adding new column in dataframe

df["MaxTemp"] = maxm[0]
df["AvgTemp"] = avg[0]
df["MinTemp"] = minm[0]

df["MaxHumid"] = maxm[1]
df["MinHumid"] = minm[1]
df["AvgHumid"] = avg[1]

df["MaxPressure"] = maxm[2]
df["AvgPressure"] = avg[2]
df["MinPressure"] = minm[2]

df["MaxSpeed"] = maxm[4]
df["AvgSpeed"] = avg[4]
df["MinSpeed"] = minm[4]

df["AvgDirn"] = avg[3]

In [18]:
#Droping unwanted columns in the dataframe

df = df.drop(["Temperature", "Humidity", "Pressure", "Wind Dirn", "Win Speed", "Weather Description"], axis = 1)

In [19]:
#Calculate max frequencey of weather description
WeatherDescription = []

for day in WeatherDiscpc:
    high = 0
    result = day[0]
    for val in day:
        freq = day.count(val)
        if freq > high:
            high = freq
            result = val
    WeatherDescription.append(result)
    
df["Weather Description"] = WeatherDescription  

In [20]:
#Creating new Column in the dataframe
def timeValue(par, time):
    value = []
    for day in par:
        for i, val in enumerate(day):
            if i == time:
                value.append(val)
    return value

for val in parameter_disc:
    hour = [6,12,20]
    for i in hour:
        col = val +str(i)
        df[col] = timeValue(parameter_disc[val], i)


In [21]:
df.reset_index(inplace = True, drop = True)

In [22]:
maxTemp_next = []
minTemp_next = []
for temp in zip(df["MaxTemp"][1:], df["MinTemp"][1:]):
    maxTemp_next.append(temp[0])
    minTemp_next.append(temp[1])
    
df =df.drop(1851)

In [23]:
df["maxTemp_next"], df["minTemp_next"] = maxTemp_next, minTemp_next

In [24]:
df.to_csv("DataSets/NewProcessed.csv")

In [25]:
df.head()

,Year,Months,Day,MaxTemp,AvgTemp,MinTemp,MaxHumid,MinHumid,AvgHumid,MaxPressure,...,WindDirn12,WindDirn20,WinSpeed6,WinSpeed12,WinSpeed20,WeatherDiscpc6,WeatherDiscpc12,WeatherDiscpc20,maxTemp_next,minTemp_next
0,2012,10,02,22.70,18.74,16.89,55.0,43.0,50.50,1013.0,...,269,240,6,5,5,3,3,24,20.73,16.39
1,2012,10,03,20.73,17.92,16.39,100.0,46.0,68.54,1017.0,...,80,60,3,1,4,0,18,1,23.12,17.97
2,2012,10,04,23.12,19.84,17.97,100.0,83.0,95.00,1018.0,...,30,40,3,2,3,4,16,16,24.12,19.43
3,2012,10,05,24.12,21.89,19.43,100.0,78.0,93.04,1020.0,...,0,160,0,0,3,4,4,6,27.63,15.63
4,2012,10,06,27.63,20.80,15.63,100.0,39.0,71.04,1018.0,...,290,260,2,1,2,4,16,3,25.69,15.86


In [27]:
df.shape

(1851, 37)

In [38]:
test_size = 100
trainSet = df[:df.shape[0]-test_size]
testSet = df[df.shape[0]-test_size:]
trainSet.to_csv("DataSets/trainSet.csv")
testSet.to_csv("DataSets/testSet.csv")